In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/CEDAR/preprocessed_signatures'
size = (220, 155)

In [ ]:
EPOCHS = 20
BS = 16

In [ ]:
import numpy as np
# import os
# import glob
# import matplotlib.pyplot as plt
import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import MobileNetV2
# from keras.applications.mobilenet import preprocess_input
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPool2D, Activation
# from tensorflow.nn import local_response_normalization
from keras.layers import Conv2D
# from tensorflow.keras.models import Model
from keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.preprocessing.image import load_img
# from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json, Sequential
# from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
X = np.load(path + '/X.npy')
y = np.load(path + '/y.npy')

In [ ]:
X = X/255.0
X = X.reshape(X.shape[0], size[0], size[1], 3)

In [ ]:
X.shape

(2640, 220, 155, 3)

In [ ]:
y1 = []
for i in range(y.shape[0]):
  y1.append([ np.argmax(y[i]) ])

y = np.array(y1)

In [ ]:
(x_train, x_test, y_train, y_test) = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 24)

In [ ]:
model = Sequential()

In [ ]:
model.add( Conv2D(filters = 96, kernel_size = 11, input_shape = (size[0], size[1], 3)) )

In [ ]:
model.add( BatchNormalization() )
model.add(Activation('relu'))

In [ ]:
model.add( MaxPool2D(pool_size = 3, strides = 2) )
model.add( Conv2D(filters = 256, kernel_size = 5) )

In [ ]:
model.add( BatchNormalization() )
model.add(Activation('relu'))

In [ ]:
model.add( MaxPool2D(pool_size = 3, strides = 2) )
model.add( Dropout(0.3) )
model.add( Conv2D(filters = 384, kernel_size = 3, activation = 'relu') )
model.add( Conv2D(filters = 256, kernel_size = 3, activation = 'relu') )
model.add( MaxPool2D(pool_size = 3, strides = 2) )
model.add( Dropout(0.3) )

model.add(Flatten())

model.add( Dense(units = 1024, activation = 'relu') )
model.add( Dropout(0.3) )
model.add( Dense(units = 1, activation = 'sigmoid') )

In [ ]:
# optimiz = Adam(learning_rate = 0.0001, weight_decay = 0.0005, epsilon = 1e-08, use_ema = True, ema_momentum = 0.9,
              #  clipnorm = None, clipvalue = None, global_clipnorm = None)

optimiz = Adam(lr = 0.0001, beta_1 = 0.9, epsilon = 1e-08, decay = 0.0005)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.compile(optimizer = optimiz, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
modelp_es = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 0)
modelp_checkpointer = ModelCheckpoint(filepath = path + "/CEDAR_signatures_siamese_1.h5", verbose = 1, save_weights_only = True)

In [ ]:
model.fit(x_train, y_train, epochs = EPOCHS, batch_size = BS, validation_data = (x_test, y_test), callbacks = [modelp_es, modelp_checkpointer])

Epoch 1/20
132/132 [==============================] - ETA: 0s - loss: 0.7221 - accuracy: 0.6586
Epoch 1: saving model to /content/drive/MyDrive/CEDAR/preprocessed_signatures/CEDAR_signatures_siamese_1.h5
132/132 [==============================] - 42s 295ms/step - loss: 0.7221 - accuracy: 0.6586 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/20
132/132 [==============================] - ETA: 0s - loss: 0.4548 - accuracy: 0.7983
Epoch 2: saving model to /content/drive/MyDrive/CEDAR/preprocessed_signatures/CEDAR_signatures_siamese_1.h5
132/132 [==============================] - 24s 181ms/step - loss: 0.4548 - accuracy: 0.7983 - val_loss: 0.6912 - val_accuracy: 0.5000
Epoch 3/20
132/132 [==============================] - ETA: 0s - loss: 0.3782 - accuracy: 0.8277
Epoch 3: saving model to /content/drive/MyDrive/CEDAR/preprocessed_signatures/CEDAR_signatures_siamese_1.h5
132/132 [==============================] - 23s 174ms/step - loss: 0.3782 - accuracy: 0.8277 - val_loss: 0.6807 - val_acc

In [ ]:
model.save(path + '/final_model_siamese.h5')

In [ ]:
eval_result = model.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_result)

17/17 [==============================] - 3s 73ms/step - loss: 0.5237 - accuracy: 0.8902
[test loss, test accuracy]: [0.5236613750457764, 0.8901515007019043]
